In [114]:
import mpmath as mp

# Start with higher precision and N, especially for higher n
mp.dps = 100 # Increased precision
N = 1000     # Increased truncation

# --- Constants and Physical Parameters ---
s = 2
l = 2

# --- Core Functions (beta corrected) ---
def alpha(n, omega):
    return n**2 + (2 - 2j*omega)*n + 1 - 2j*omega

def beta(n, omega):
    return -(2*n**2 + (2 - 8j*omega)*n - 8*omega**2 - 4j*omega + l*(l+1) + 1 - s**2)

def gamma(n, omega):
    return n**2 - 4j*omega*n - 4*omega**2 - s**2

def R_N_series(omega, N):
    """
    Corrected Nollert asymptotic expansion.
    1. Fixed C1 sign choice (using .real attribute).
    2. Removed the unstable C3 term.
    """
    C0 = -1.0
    
    C1 = mp.sqrt(-2j*omega)
    if C1.real > 0: # Nollert's condition: Re(C1) < 0
        C1 = -C1
        
    C2 = 0.75 + 2j*omega
    
    # C3 term is removed for stability.
    # We compensate by using a larger N.
    return C0 + C1/mp.sqrt(N) + C2/N

def continued_fraction_fixedN(omega):
    """The function whose root we want to find."""
    R = R_N_series(omega, N)
    for n in reversed(range(N)):
        R = gamma(n+1, omega) / (beta(n+1, omega) - alpha(n+1, omega) * R)
    
    # The final equation is 0 = beta_0 - alpha_0 * R_0
    return beta(0, omega) - alpha(0, omega) * R

# --- Root Finding with Highly Accurate Guesses ---

# High-quality initial guesses for s=2, l=2 in 2M units
# (Sourced from Berti, Cardoso, Will (2009) and multiplied by 2)
omega_guesses = [
    0.747343 - 0.177925j, # n=0
    0.693422 - 0.547830j, # n=1
    0.602107 - 0.956554j, # n=2
    0.503010 - 1.410296j, # n=3
    0.415029 - 1.893690j, # n=4
    0.338599 - 2.391216j, # n=5
]
print(f"Finding QNMs for s={s}, l={l} with N={N} and dps={mp.dps}\n")

# Loop through the guesses to find each overtone
for n, omega_guess in enumerate(omega_guesses):
    print(f"Searching for overtone n={n} with guess: {omega_guess}")
    try:
        # Using 'mnewton' (Newton's method) is often faster and more
        # stable than 'muller' if the function is smooth, which this is.
        omega_qnm = mp.findroot(
            continued_fraction_fixedN, 
            omega_guess, 
            solver='mnewton'
        )
        print(f"  └── Found QNM: {omega_qnm}\n")
    except ValueError:
        print(f"  └── Solver could not find a root for guess: {omega_guess}\n")

Finding QNMs for s=2, l=2 with N=1000 and dps=100

Searching for overtone n=0 with guess: (0.747343-0.177925j)
  └── Found QNM: (0.74734336883608367158698400595409654037078951677937120801361020371771168281871289 - 0.17792463137787139656092185436904515639381341245604675949827667402183632697588839j)

Searching for overtone n=1 with guess: (0.693422-0.54783j)
  └── Found QNM: (0.69342199375832687943535071946078303568861068266505994819555659232533539375105671 - 0.54782975058246963469912044427149028263579228099398316461485090030866750575687338j)

Searching for overtone n=2 with guess: (0.602107-0.956554j)
  └── Found QNM: (0.60210690922473278761191285547720082365080168433296138798058569337101886170944257 - 0.95655396644614361996702665524255922687920368962214920446176812471491744059921089j)

Searching for overtone n=3 with guess: (0.50301-1.410296j)
  └── Found QNM: (0.50300992437120732429024973845128413501195659208200185362102121315525738616681153 - 1.410296404867012010622857094680251160833